##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo crear una canalización de TFX para sus datos con la plantilla Penguin

---


Nota: Recomendamos ejecutar este tutorial en [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench) de Google Cloud. [Vaya a Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench).

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

## Introducción

Este documento nos ofrece instrucciones para crear una canalización de TensorFlow Extended (TFX) para su propio conjunto de datos con ayuda de la *plantilla Penguin* que se proporciona con el paquete de Python para TFX. La canalización creada usará inicialmente el conjunto de datos de [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html), pero transformaremos la canalización para su conjunto de datos.


### Requisitos previos

- Linux / MacOS
- Python 3.6-3.8
- Bloc de notas Jupyter


## Paso 1. Copie la plantilla predefinida en el directorio de su proyecto

En este paso, crearemos un directorio y archivos de proyecto de canalización de trabajo copiando archivos de la *plantilla Penguin* en TFX. Puede considerar esto como un andamio para su proyecto de canalización de TFX.

### Actualización de pip

Si estamos ejecutando Colab, debemos asegurarnos de tener la última versión de Pip. Por supuesto, los sistemas locales se pueden actualizar por separado.

Nota: La actualización probablemente también sea una buena idea si está ejecutando Vertex AI Workbench.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### Instalación del paquete requerido

Primero, instale TFX y TensorFlow Model Analysis (TFMA).


In [ ]:
!pip install -U tfx tensorflow-model-analysis

Revisemos las versiones de TFX.

In [ ]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tfx

print('TF version: {}'.format(tf.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFX version: {}'.format(tfx.__version__))

Estamos listos para crear una canalización.

Establezca `PROJECT_DIR` en el destino apropiado para su entorno. El valor predeterminado es `~/imported/${PIPELINE_NAME}`, que es apropiado para el entorno de [bloc de notas de Google Cloud AI Platform](https://console.cloud.google.com/ai-platform/notebooks/).

Puede cambiar el nombre de su canalización si cambia el valor de `PIPELINE_NAME` a continuación. Este también se convertirá en el nombre del directorio del proyecto donde se colocarán sus archivos.


In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Set this project directory to your new tfx pipeline project.
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "imported", PIPELINE_NAME)

### Copia de archivos de plantilla

TFX incluye la plantilla `penguin` con el paquete de Python para TFX. La plantilla `penguin` contiene muchas instrucciones para incorporar su conjunto de datos al proceso, que es el propósito de este tutorial.

El comando de CLI `tfx template copy` copia archivos de plantilla predefinidos al directorio de su proyecto.

In [ ]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=penguin

Cambie el contexto del directorio de trabajo en este bloc de notas al directorio del proyecto.

In [ ]:
%cd {PROJECT_DIR}

> NOTA: Si está utilizando un bloc de notas de JupyterLab o Google Cloud AI Platform, no olvide cambiar el directorio en `File Browser` a la izquierda haciendo clic en el directorio del proyecto una vez que lo haya creado.

### Exploración de archivos fuente copiados

La plantilla TFX proporciona archivos de estructura básicos para compilar una canalización, incluido el código fuente de Python y datos de muestra. La plantilla `penguin` usa el mismo conjunto de datos *Palmer Penguins* y modelo de ML que el [ejemplo de Penguin](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

Esta es una breve introducción a cada uno de los archivos de Python.

- `pipeline`: este directorio contiene la definición de la canalización.
    - `configs.py`: define constantes comunes para los ejecutores de la canalización
    - `pipeline.py`: define los componentes de TFX y una canalización
- `models`: este directorio contiene definiciones de modelos de ML
    - `features.py`, `features_test.py`: define características para el modelo
    - `preprocessing.py`, `preprocessing_test.py`: define rutinas de preprocesamiento de datos
    - `constants.py`: define las constantes del modelo
    - `model.py`, `model_test.py`: define el modelo de ML a partir de marcos de ML como TensorFlow
- `local_runner.py`: define un ejecutor para el entorno local que emplea el motor de orquestación local
- `kubeflow_runner.py`: define un ejecutor para el motor de orquestación de Kubeflow Pipelines


De forma predeterminada, la plantilla solo incluye componentes de TFX estándar. Si necesita algunas acciones personalizadas, puede crear componentes personalizados para su canalización. Consulte la [guía de componentes personalizados de TFX](https://www.tensorflow.org/tfx/guide/understanding_custom_components) para obtener más detalles.

#### Archivos de prueba unitaria

Quizás note que hay algunos archivos con `_test.py` en su nombre. Estas son pruebas unitarias de la canalización y se recomienda agregar más pruebas unitarias a medida que implemente sus propias canalizaciones. Puede ejecutar pruebas unitarias si proporciona el nombre del módulo de los archivos de prueba con la marca `-m`. Generalmente puede obtener el nombre de un módulo al eliminar la extensión `.py` y reemplazarla por `/` con `.`. Por ejemplo:

In [ ]:
import sys
!{sys.executable} -m models.features_test

### Creación de una canalización de TFX en el entorno local

TFX admite varios motores de orquestación para ejecutar canalizaciones. Usaremos un motor de orquestación local. El motor de orquestación local se ejecuta sin más dependencias y es adecuado para el desarrollo y la depuración porque se ejecuta en un entorno local en lugar de depender de clústeres informáticos remotos.

Usaremos `local_runner.py` para ejecutar su canalización usando el orquestador local. Tiene que crear una canalización antes de ejecutarla. Puede crear una canalización con el comando `pipeline create`.


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

El comando `pipeline create` registra la canalización definida en `local_runner.py` sin ejecutarla realmente.

Ejecutará la canalización creada con el comando `run create` en los siguientes pasos.


## Paso 2. Ingiera SUS datos a la canalización

La canalización inicial ingiere el conjunto de datos Penguin que se incluye en la plantilla. Debe colocar sus datos en la canalización, y la mayoría de las canalizaciones de TFX comienzan con el componente ExampleGen.

### Elección de un tipo de ExampleGen

Sus datos se pueden almacenar en cualquier lugar al que pueda acceder su canalización, ya sea en un sistema de archivos local o distribuido, o en un sistema con capacidad de consulta. TFX ofrece varios [componentes `ExampleGen`](https://www.tensorflow.org/tfx/guide/examplegen) para llevar sus datos a una canalización de TFX. Puede elegir uno de los siguientes componentes de generación de ejemplo.

- CsvExampleGen: lee archivos en formato CSV en un directorio. Se usa en el [ejemplo de pingüinos](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin) y en el [ejemplo de taxis de Chicago](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline).
- ImportExampleGen: toma archivos TFRecord con formato de datos TF Example. Se usa en [ejemplos MNIST](https://github.com/tensorflow/tfx/tree/master/tfx/examples/mnist).
- FileBasedExampleGen para formato [Avro](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/avro_executor.py) o [Parquet](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/parquet_executor.py).
- [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen): lee datos directamente en Google Cloud BigQuery. Se usa en [ejemplos de taxis de Chicago](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline).

También puede crear su propio ExampleGen, por ejemplo, tfx incluye [un ExecampleGen personalizado que utiliza Presto](https://github.com/tensorflow/tfx/tree/master/tfx/examples/custom_components/presto_example_gen) como fuente de datos. Consulte la [guía](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen) para obtener más información sobre cómo usar y desarrollar ejecutores personalizados.

Una vez que decida qué tipo de ExampleGen usar, deberá modificar la definición de canalización para usar sus datos.

1. Modifique `DATA_PATH` en `local_runner.py` y configúrelo en la ubicación de sus archivos.

- Si tiene archivos en el entorno local, especifique la ruta. Esta es la mejor opción para desarrollar o depurar una canalización.
- Si los archivos están almacenados en GCS, puede usar una ruta que comience con `gs://{bucket_name}/...` Asegúrese de poder acceder a GCS desde su terminal, por ejemplo, usando [`gsutil`](https://cloud.google.com/storage/docs/gsutil). Siga la [guía de autorización en Google Cloud](https://cloud.google.com/sdk/docs/authorizing) si es necesario.
- Si quiere usar un ExampleGen basado en consultas como BigQueryExampleGen, necesita una instrucción de consulta para seleccionar datos de la fuente de datos. Hay algunas cosas más que se deben configurar para usar Google Cloud BigQuery como fuente de datos.
    - En `pipeline/configs.py`:
        - Cambie `GOOGLE_CLOUD_PROJECT` y `GCS_BUCKET_NAME` por su proyecto de GCP y el nombre del depósito. El depósito debería existir antes de ejecutar la canalización.
        - Descomente la variable `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS`.
        - Descomente y configure la variable `BIG_QUERY_QUERY` en **su declaración de consulta**.
    - En `local_runner.py`:
        - Comente el argumento `data_path` y descomente el argumento `query` en `pipeline.create_pipeline()`.
    - En `pipeline/pipeline.py`:
        - Comente el argumento `data_path` y descomente el argumento `query` en `create_pipeline()`.
        - Use [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen) en lugar de CsvExampleGen.

1. Reemplace el CsvExampleGen existente por su clase ExampleGen en `pipeline/pipeline.py`. Cada clase de ExampleGen tiene una firma diferente. Consulte la [guía de componentes de ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) para obtener más detalles. No olvide importar los módulos necesarios con declaraciones `import` en `pipeline/pipeline.py`.

La canalización inicial consta de cuatro componentes: `ExampleGen`, `StatisticsGen`, `SchemaGen` y `ExampleValidator`. No necesitamos cambiar nada para `StatisticsGen`, `SchemaGen` y `ExampleValidator`. Ejecutemos la canalización por primera vez.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Debería ver el mensaje "Component ExampleValidator is finished." si la canalización se ejecutó correctamente.

### Análisis de la salida de la canalización

La canalización de TFX produce dos tipos de resultados, artefactos y una [base de datos de metadatos (MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) que contiene metadatos de artefactos y ejecuciones de canalizaciones. La ubicación de la salida se define en `local_runner.py`. De forma predeterminada, los artefactos se almacenan en el directorio `tfx_pipeline_output` y los metadatos se almacenan como una base de datos sqlite en el directorio `tfx_metadata`.

Puede utilizar las API de MLMD para examinar estos resultados. Primero, definiremos algunas funciones de utilidad para buscar artefactos de salida que se acaban de producir.

In [ ]:
import tensorflow as tf
import tfx
from ml_metadata import errors
from ml_metadata.proto import metadata_store_pb2
from tfx.types import artifact_utils

# TODO(b/171447278): Move these functions into TFX library.

def get_latest_executions(store, pipeline_name, component_id = None):
  """Fetch all pipeline runs."""
  if component_id is None:  # Find entire pipeline runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('run')
        if c.properties['pipeline_name'].string_value == pipeline_name
    ]
  else:  # Find specific component runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('component_run')
        if c.properties['pipeline_name'].string_value == pipeline_name and
           c.properties['component_id'].string_value == component_id
    ]
  if not run_contexts:
    return []
  # Pick the latest run context.
  latest_context = max(run_contexts,
                       key=lambda c: c.last_update_time_since_epoch)
  return store.get_executions_by_context(latest_context.id)

def get_latest_artifacts(store, pipeline_name, component_id = None):
  """Fetch all artifacts from latest pipeline execution."""
  executions = get_latest_executions(store, pipeline_name, component_id)

  # Fetch all artifacts produced from the given executions.
  execution_ids = [e.id for e in executions]
  events = store.get_events_by_execution_ids(execution_ids)
  artifact_ids = [
      event.artifact_id for event in events
      if event.type == metadata_store_pb2.Event.OUTPUT
  ]
  return store.get_artifacts_by_id(artifact_ids)

def find_latest_artifacts_by_type(store, artifacts, artifact_type):
  """Get the latest artifacts of a specified type."""
  # Get type information from MLMD
  try:
    artifact_type = store.get_artifact_type(artifact_type)
  except errors.NotFoundError:
    return []
  # Filter artifacts with type.
  filtered_artifacts = [aritfact for aritfact in artifacts
                        if aritfact.type_id == artifact_type.id]
  # Convert MLMD artifact data into TFX Artifact instances.
  return [artifact_utils.deserialize_artifact(artifact_type, artifact)
      for artifact in filtered_artifacts]


from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

import pprint

from tfx.orchestration import metadata
from tfx.types import artifact_utils
from tfx.types import standard_artifacts

def preview_examples(artifacts):
  """Preview a few records from Examples artifacts."""
  pp = pprint.PrettyPrinter()
  for artifact in artifacts:
    print("==== Examples artifact:{}({})".format(artifact.name, artifact.uri))
    for split in artifact_utils.decode_split_names(artifact.split_names):
      print("==== Reading from split:{}".format(split))
      split_uri = artifact_utils.get_split_uri([artifact], split)

      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = [os.path.join(split_uri, name)
                            for name in os.listdir(split_uri)]
      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames,
                                        compression_type="GZIP")
      # Iterate over the first 2 records and decode them.
      for tfrecord in dataset.take(2):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        pp.pprint(example)

import local_runner

metadata_connection_config = metadata.sqlite_metadata_connection_config(
              local_runner.METADATA_PATH)

Ahora podemos leer metadatos de artefactos de salida de MLMD.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous pipeline run.
    artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
    # Find artifacts of Examples type.
    examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)
    # Find artifacts generated from StatisticsGen.
    stats_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleStatistics.TYPE_NAME)
    # Find artifacts generated from SchemaGen.
    schema_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Schema.TYPE_NAME)
    # Find artifacts generated from ExampleValidator.
    anomalies_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleAnomalies.TYPE_NAME)

Ahora podemos examinar los resultados de cada componente. [La validación de datos de Tensorflow (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) se usa en `StatisticsGen`, `SchemaGen` y `ExampleValidator`, y TFDV se puede usar para visualizar los resultados de estos componentes.

En este tutorial, usaremos métodos auxiliares de visualización en TFX que usan TFDV internamente para mostrar la visualización. Consulte el [tutorial de componentes de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) para obtener más información sobre cada componente.

#### Análisis del formulario de salida ExampleGen

Examinemos el resultado de ExampleGen. Echemos un vistazo a los dos primeros ejemplos de cada división:

In [ ]:
preview_examples(examples_artifacts)

De forma predeterminada, TFX ExampleGen divide los ejemplos en dos divisiones, *train* y *eval*, pero puede [ajustar la configuración de su división](https://www.tensorflow.org/tfx/guide/examplegen#span_version_and_split).

#### Análisis del formulario de salida de StatisticsGen


In [ ]:
visualize_artifacts(stats_artifacts)

Estas estadísticas se proporcionan a SchemaGen para construir un esquema de datos automáticamente.

#### Análisis del formulario de salida de SchemaGen


In [ ]:
visualize_artifacts(schema_artifacts)

Este esquema se infiere automáticamente a partir del resultado de StatisticsGen. Usaremos este esquema generado en este tutorial, pero también puede [modificar y personalizar el esquema](https://www.tensorflow.org/tfx/guide/statsgen#creating_a_curated_schema).

#### Análisis del formulario de salida de ExampleValidator


In [ ]:
visualize_artifacts(anomalies_artifacts)

Si se encuentra alguna anomalía, puede revisar los datos para comprobar que todos los ejemplos siguen sus suposiciones. Los resultados de otros componentes como StatisticsGen pueden ser útiles. Las anomalías encontradas no bloquean la ejecución de la canalización.

Puede ver las funciones disponibles en los resultados de `SchemaGen`. Si sus funciones se pueden usar para construir el modelo de ML directamente en `Trainer`, puede omitir el siguiente paso e ir al Paso 4. De lo contrario, puede realizar algún trabajo de ingeniería de funciones en el siguiente paso. El componente `Transform` es necesario cuando se requieren operaciones de paso completo, como calcular promedios, especialmente cuando es necesario escalar.

## Paso 3. (Opcional) Ingeniería de características con el componente Transform

En este paso, definirá varios trabajos de ingeniería de funciones que utilizará el componente `Transform` en la canalización. Consulte la [guía de componentes Transform](https://www.tensorflow.org/tfx/guide/transform) para obtener más información.

Esto solo es necesario si su código de entrenamiento requiere funciones adicionales que no estén disponibles en la salida de ExampleGen. De lo contrario, siéntase libre de avanzar rápidamente al siguiente paso del uso de Trainer.

### Definición de las características del modelo

`models/features.py` contiene constantes para definir características para el modelo, incluidos los nombres de las características, el tamaño del vocabulario, etc. De forma predeterminada, la plantilla `penguin` tiene dos costantes, `FEATURE_KEYS` y `LABEL_KEY`, porque nuestro modelo `penguin` resuelve un problema de clasificación mediante aprendizaje supervisado y todas las características son características numéricas continuas. Consulte las [definiciones de funciones del ejemplo de taxis de Chicago](https://github.com/tensorflow/tfx/blob/master/tfx/experimental/templates/taxi/models/features.py) para ver otro ejemplo.


### Implementación del preprocesamiento para entrenamiento/servicio en preprocessing_fn().

La ingeniería de características real ocurre en la función `preprocessing_fn()` en `models/preprocessing.py`.

En `preprocessing_fn` puede definir una serie de funciones que manipulan el dictado de entrada de los tensores para producir el dictado de salida de los tensores. Hay funciones ayudantes como `scale_to_0_1` y `compute_and_apply_vocabulary` en la API de TensorFlow Transform o simplemente puede usar funciones normales de TensorFlow. De forma predeterminada, la plantilla `penguin` incluye usos de ejemplo de la función [tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score) para normalizar los valores de las características.

Consulte la [guía de Tensorflow Transform](https://www.tensorflow.org/tfx/transform/get_started) para obtener más información sobre la creación de `preprocessing_fn`.


### Incorporación del componente Transform a la canalización

Si su preprocessing_fn está lista, agregue el componente `Transform` a la canalización.

1. En el archivo `pipeline/pipeline.py`, descomente `# components.append(transform)` para agregar el componente a la canalización.

Puede actualizar la canalización y ejecutarla nuevamente.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Si la canalización se ejecutó correctamente, debería ver el mensaje "Component Transform is finished". en *algún lugar* del registro. Como el componente `Transform` y el componente `ExampleValidator` no dependen entre sí, el orden de ejecución no es fijo. Dicho esto, `Transform` y `ExampleValidator` pueden ser el último componente en la ejecución del proceso.

### Análisis del formulario de salida de Transform

El componente Transform crea dos tipos de resultados, un grafo de Tensorflow y ejemplos transformados. Los ejemplos transformados son del tipo de artefacto Ejemplos que también produce ExampleGen, pero este contiene valores de características transformados.

Puede examinarlos como lo hicimos en el paso anterior.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous run of Transform component.
    artifacts = get_latest_artifacts(metadata_handler.store,
                                     PIPELINE_NAME, "Transform")
    # Find artifacts of Examples type.
    transformed_examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)

In [ ]:
preview_examples(transformed_examples_artifacts)

## Paso 4. Entrene su modelo con el componente Trainer

Compilaremos un modelo de ML a partir del componente `Trainer`. Consulte la [guía del componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para obtener más información. Debe proporcionar el código de su modelo al componente Trainer.

### Definición del modelo

En la plantilla Penguin, `models.model.run_fn` sirve de argumento `run_fn` para el componente `Trainer`. Esto significa que la función `run_fn()` en `models/model.py` se llamará cuando se ejecute el componente `Trainer`. Puede ver el código para construir un modelo DNN simple con la API `keras` en el código proporcionado. Consulte la guía de [TensorFlow 2.x en TFX](https://www.tensorflow.org/tfx/guide/keras) para obtener más información sobre el uso de la API de keras en TFX.

En esta `run_fn`, debe compilar un modelo y guardarlo en un directorio señalado por `fn_args.serving_model_dir` que está especificado por el componente. Puede usar otros argumentos en `fn_args` que se pasa a `run_fn`. Consulte los [códigos relacionados](https://github.com/tensorflow/tfx/blob/b01482442891a49a1487c67047e85ab971717b75/tfx/components/trainer/executor.py#L141) para obtener la lista completa de argumentos en `fn_args`.

Defina sus funciones en `models/features.py` y utilícelas según sea necesario. Si transformó sus funciones en el Paso 3, debe usar características transformadas como entradas para su modelo.

### Incorporación del componente Trainer a la canalización

Si su run_fn está lista, agregue el componente `Trainer` a la canalización.

1. En el archivo `pipeline/pipeline.py`, descomente `# components.append(trainer)` para agregar el componente a la canalización.

Los argumentos para el componente Trainer pueden depender de si usa o no el componente Transform.

- Si **NO** usa el componente `Transform`, no es necesario que cambie los argumentos.

- Si usa el componente `Transform`, tendrá que cambiar los argumentos al crear una instancia del componente `Trainer`.

    - Cambie el argumento `examples` a `examples=transform.outputs['transformed_examples'],`. Debemos usar ejemplos transformados para el entrenamiento.
    - Agregue el argumento `transform_graph` como `transform_graph=transform.outputs['transform_graph'],` Este grafo contiene un grafo de TensorFlow para las operaciones de transformación.
    - Después de aplicar los cambios anteriores, el código para la creación del componente Trainer tendrá el siguiente aspecto.

    ```python
    # If you use a Transform component.
    trainer = Trainer(
        run_fn=run_fn,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        ...
    ```

Puede actualizar la canalización y ejecutarla nuevamente.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Cuando esta ejecución se ejecute correctamente, habrá creado y ejecutado su primera canalización de TFX para su modelo. ¡Felicidades!

Su nuevo modelo se ubicará en algún lugar debajo del directorio de salida, pero sería mejor tener un modelo en una ubicación fija o en servicio fuera de la canalización de TFX que contenga muchos resultados provisionales. Aún mejor con la evaluación continua del modelo construido, que es fundamental en los sistemas de producción de ML. Veremos cómo funcionan la evaluación continua y las implementaciones en TFX en el siguiente paso.

## Paso 5. (Opcional) Evalúe el modelo con Evaluator y publíquelo con Pusher


El componente [`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) evalúa continuamente cada modelo creado desde `Trainer` y [`Pusher`](https://www.tensorflow.org/tfx/guide/pusher) copia el modelo en una ubicación predefinida en el sistema de archivos o incluso en [modelos de Google Cloud AI Platform](https://console.cloud.google.com/ai-platform/models).

### Incorporación del componente Evaluator a la canalización

En el archivo `pipeline/pipeline.py`:

1. Descomente `# components.append(model_resolver)` para agregar el solucionador de modelos más reciente a la canalización. Evaluator sirve para comparar un modelo con el modelo de referencia anterior que pasó Evaluator en la última ejecución del proceso. `LatestBlessedModelResolver` encuentra el último modelo que pasó Evaluator.
2. Configure `tfma.MetricsSpec` adecuadas para su modelo. La evaluación puede ser diferente para cada modelo de ML. En la plantilla Penguin, se usó `SparseCategoricalAccuracy` porque estamos resolviendo un problema de clasificación de múltiples categorías. También debe especificar `tfma.SliceSpec` para analizar su modelo en busca de segmentos específicos. Para obtener más detalles, consulte la [guía del componente Evaluator](https://www.tensorflow.org/tfx/guide/evaluator).
3. Descomente `# components.append(evaluator)` para agregar el componente a la canalización.

Puede actualizar la canalización y ejecutarla nuevamente.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

### Análisis de la salida de Evaluator

Este paso requiere la extensión del bloc de notas Jupyter de TensorFlow Model Analysis (TFMA). Tenga en cuenta que la versión de la extensión del bloc de notas de TFMA debe ser idéntica a la versión del paquete de Python para TFMA.

El siguiente comando instalará la extensión del bloc de notas de TFMA desde el registro NPM. Es posible que tarde varios minutos en completarse.

In [ ]:
# Install TFMA notebook extension.
!jupyter labextension install tensorflow_model_analysis@{tfma.__version__}

Si se completa la instalación, **vuelva a cargar su navegador** para que se aplique la extensión.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
  # Search all aritfacts from the previous pipeline run.
  artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
  model_evaluation_artifacts = find_latest_artifacts_by_type(
      metadata_handler.store, artifacts,
      standard_artifacts.ModelEvaluation.TYPE_NAME)

In [ ]:
if model_evaluation_artifacts:
  tfma_result = tfma.load_eval_result(model_evaluation_artifacts[0].uri)
  tfma.view.render_slicing_metrics(tfma_result)

### Incorporación del componente Pusher a la canalización

Si el modelo se ve prometedor, debemos publicarlo. El [componente Pusher](https://www.tensorflow.org/tfx/guide/pusher) puede publicar el modelo en una ubicación del sistema de archivos o en modelos GCP AI Platform mediante [un ejecutor personalizado](https://github.com/tensorflow/tfx/blob/master/tfx/extensions/google_cloud_ai_platform/pusher/executor.py).

El componente <a><code data-md-type="codespan">Evaluator</code></a> evalúa continuamente cada modelo creado desde <code>Trainer</code> y <a><code>Pusher</code></a> copia el modelo en una ubicación predefinida en el sistema de archivos o incluso en <a>modelos de Google Cloud AI Platform</a>.

1. En `local_runner.py`, configure `SERVING_MODEL_DIR` en un directorio para publicar.
2. En el archivo `pipeline/pipeline.py`, descomente `# components.append(pusher)` para agregar Pusher a la canalización.

Puede actualizar la canalización y ejecutarla nuevamente.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Debería poder encontrar su nuevo modelo en `SERVING_MODEL_DIR`.

## Paso 6. (Opcional) Implemente su canalización en Kubeflow Pipelines en GCP


Como se mencionó anteriormente, `local_runner.py` es bueno para fines de depuración o desarrollo, pero no es la mejor solución para cargas de trabajo de producción. En este paso, implementaremos la canalización en Kubeflow Pipelines en Google Cloud.

### Preparación

Necesitamos el paquete de Python para `kfp` y el programa `skaffold` para implementar una canalización en un clúster de Kubeflow Pipelines.

In [ ]:
!pip install --upgrade -q kfp

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold

Debe mover el binario `skaffold` al lugar donde su shell pueda encontrarlo. O puede especificar la ruta a skaffold cuando ejecute el binario `tfx` con la marca `--skaffold-cmd`.

In [ ]:
# Move skaffold binary into your path
!mv skaffold /home/jupyter/.local/bin/

También necesita un clúster de Kubeflow Pipelines para ejecutar la canalización. Siga los pasos 1 y 2 del [tutorial de TFX en Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

Cuando su clúster esté listo, abra el panel de canalizaciones al hacer clic en *Abrir panel de canalizaciones* en la [página `Pipelines` de la consola en la nube de Google](http://console.cloud.google.com/ai-platform/pipelines). La URL de esta página es `ENDPOINT` para solicitar una ejecución de canalización. El valor del punto de conexión es todo lo que está en la URL después de https://, hasta googleusercontent.com inclusive. Coloque su punto de conexión en el siguiente bloque de código.


In [ ]:
ENDPOINT='' # Enter your ENDPOINT here.

Para ejecutar nuestro código en un clúster de Kubeflow Pipelines, necesitamos empaquetar nuestro código en una imagen de contenedor. La imagen se creará automáticamente durante la implementación de nuestra canalización y solo necesitará establecer un nombre y un registro de contenedor para su imagen. En nuestro ejemplo, usaremos el [registro de Google Container](https://cloud.google.com/container-registry) y lo llamaremos `tfx-pipeline`.

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]

# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

### Configuración de la ubicación de los datos

Se debe poder acceder a sus datos desde el clúster de Kubeflow Pipelines. Si ha usado datos en su entorno local, es posible que deba cargarlos en un almacenamiento remoto como Google Cloud Storage. Por ejemplo, podemos cargar datos de Penguin en un depósito predeterminado que se crea automáticamente cuando se implementa un clúster de Kubeflow Pipelines como se muestra a continuación.

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/penguin/

Actualice la ubicación de los datos almacenados en `DATA_PATH` en `kubeflow_runner.py`.

Si usa BigQueryExampleGen, no es necesario cargar el archivo de datos, pero asegúrese de que `kubeflow_runner.py` use la misma `query` y el mismo argumento `beam_pipeline_args` para la función `pipeline.create_pipeline()`.

### Implementación de la canalización

Si todo está listo, puede crear una canalización a partir del comando `tfx pipeline create`.

> Nota: Al crear una canalización para Kubeflow Pipelines, necesitamos una imagen de contenedor que se utilizará para ejecutar nuestra canalización. Y `skaffold` compilará la imagen para nosotros. Debido a que `skaffold` extrae imágenes base del Docker Hub, demorará entre 5 y 10 minutos cuando compilemos la imagen por primera vez, pero tomará mucho menos tiempo desde la segunda compilación.


In [ ]:
!tfx pipeline create  \
--engine=kubeflow \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

Ahora inicie una ejecución con la canalización recién creada con ayuda del comando `tfx run create`.

In [ ]:
!tfx run create --engine=kubeflow --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

O también puede ejecutar la canalización en el panel de Kubeflow Pipelines. La nueva ejecución aparecerá en `Experiments` en el panel de Kubeflow Pipelines. Al hacer clic en el experimento, podrá monitorear el progreso y visualizar los artefactos creados durante la ejecución.

Si le interesa ejecutar su canalización en Kubeflow Pipelines, consulte instrucciones adicionales en el [tutorial de TFX en Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

### Limpieza

Para limpiar todos los recursos de Google Cloud utilizados en este paso, puede [eliminar el proyecto de Google Cloud](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) que utilizó para el tutorial.

Alternativamente, puedes limpiar recursos individuales si visita cada consola:

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)